In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
from debiaswe.data import load_gender_seed

import torch

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
print(E)
words = E.words
print("Words:", len(words))

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Words: 26423
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# gender direction
v_gender = E.diff('she', 'he')

# Uncomment below for direction based on multiple definitional pairs
# with open('./data/definitional_pairs.json', "r") as f:
#     defs = json.load(f)
# v_gender = we.doPCA(defs, E).components_[0]

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, thresh=1)

# for (a,b,c) in a_gender:
#     print(a+"-"+b)
we.viz(a_gender)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
(135018, 300)
   0                          she | he                           1.00
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                        woman | man                          0.75
   4                     daughter | son                          0.67
   5                businesswoman | businessman                  0.65
   6                         girl | boy                          0.65
   7                      actress | actor                        0.65
   8                   chairwoman | chairman                     0.63
   9                      heroine | hero                         0.62
  10                       mother | father                       0.60
  11                  spokeswoman | spokesman                    0.59
  12                       sister | brother                      0.59
  13               

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.2075867, 'skipper'),
  (-0.20267203, 'protege'),
  (-0.2020676, 'businessman'),
  (-0.19492391, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.17289859, 'captain'),
  (-0.16785556, 'architect'),
  (-0.16702037, 'financier'),
  (-0.1631364, 'warrior'),
  (-0.15280864, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637241, 'magician'),
  (-0.14401692, 'fighter_pilot'),
  (-0.13782284, 'boss'),
  (-0.13718201, 'industrialist'),
  (-0.13684887, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560377, 'stylist'),
  (0.22363171, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125953, 'nun'),
  (0.24782579, 'nanny'),
  (0.24929334, 'hairdresser'),
  (0.24946159, 'paralegal'),
  (0.25276467, 'ballerina'),
  (0.2571882, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317625, 'receptionist'),
  (0.2754029, 'waitress'),
  (

## Find soft debias transform using SGD
This code is largely based on code from https://github.com/TManzini/DebiasMulticlassWordEmbedding.

In [28]:
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
with open('./data/gender_specific_seed.json', "r") as f:
    gender_words = json.load(f)
    
W = torch.from_numpy(E.vecs).t()
print(f"Word embeddings: {W.shape}")

neutrals = list(set(words) - set(gender_words))
neutrals = torch.tensor([E.vecs[E.index[w]] for w in neutrals]).t()
print(f"Neutral embeddings: {neutrals.shape}")

gender_direction = torch.tensor([we.doPCA(defs, E).components_[0]]).t()
print(f"Gender subspace: {gender_direction.shape}")

Word embeddings: torch.Size([300, 26423])
Neutral embeddings: torch.Size([300, 26205])
Gender subspace: torch.Size([300, 1])


In [8]:
l = 0.2 # lambda

u, s, _ = torch.svd(W)
s = torch.diag(s)

# precompute
t1 = s.mm(u.t())
t2 = u.mm(s)

transform = torch.randn(300, 300, requires_grad=True)

epochs = 2000
optimizer = torch.optim.Adam([transform], lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1000,1500,1800], gamma=0.1)
best = (None, float("inf")) # (best transform, lowest loss)

for i in range(epochs):
    optimizer.zero_grad()
    
    TtT = torch.mm(transform.t(), transform)
    norm1 = (t1.mm(TtT - torch.eye(300)).mm(t2)).norm(p="fro")

    norm2 = (neutrals.t().mm(TtT).mm(gender_direction)).norm(p="fro")

    loss = norm1 + l * norm2
    if loss.item() < best[1]:
        best = (transform, loss.item())

    loss.backward()
    optimizer.step()
    scheduler.step()
    
    if i % 10 == 0:
        print("Loss @ Epoch #" + str(i) + ":", loss.item())
        
transform = best[0].detach()
print(f"Lowest loss: {best[1]}")

Loss @ Epoch #0: 927784.4375
Loss @ Epoch #10: 609690.75
Loss @ Epoch #20: 509598.0
Loss @ Epoch #30: 437633.625
Loss @ Epoch #40: 381144.8125
Loss @ Epoch #50: 335227.84375
Loss @ Epoch #60: 296960.3125
Loss @ Epoch #70: 264481.75
Loss @ Epoch #80: 236542.8125
Loss @ Epoch #90: 212263.359375
Loss @ Epoch #100: 190999.890625
Loss @ Epoch #110: 172264.4375
Loss @ Epoch #120: 155673.484375
Loss @ Epoch #130: 140920.5
Loss @ Epoch #140: 127755.4921875
Loss @ Epoch #150: 115970.6953125
Loss @ Epoch #160: 105392.59375
Loss @ Epoch #170: 95874.8046875
Loss @ Epoch #180: 87292.3203125
Loss @ Epoch #190: 79538.359375
Loss @ Epoch #200: 72520.546875
Loss @ Epoch #210: 66159.2109375
Loss @ Epoch #220: 60384.66796875
Loss @ Epoch #230: 55136.10546875
Loss @ Epoch #240: 50360.23828125
Loss @ Epoch #250: 46010.0703125
Loss @ Epoch #260: 42043.99609375
Loss @ Epoch #270: 38425.18359375
Loss @ Epoch #280: 35120.8125
Loss @ Epoch #290: 32101.640625
Loss @ Epoch #300: 29341.44140625
Loss @ Epoch #310: 

In [29]:
debiased_embeds = transform.mm(W).t().numpy()
debiased_embeds = debiased_embeds / np.linalg.norm(debiased_embeds, axis=1)[:, None]
E.vecs = debiased_embeds

a_gender_debiased = E.best_analogies_dist_thresh(v_gender)
we.viz(a_gender_debiased)

   0                          she | he                           1.00
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                        woman | man                          0.75
   4                     daughter | son                          0.67
   5                businesswoman | businessman                  0.65
   6                         girl | boy                          0.65
   7                      actress | actor                        0.65
   8                   chairwoman | chairman                     0.63
   9                      heroine | hero                         0.62
  10                       mother | father                       0.60
  11                  spokeswoman | spokesman                    0.59
  12                       sister | brother                      0.59
  13                        girls | boys                         0.59
  14                

In [10]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.12139304, 'freelance_writer'),
  (-0.12024328, 'paralegal'),
  (-0.11569916, 'lecturer'),
  (-0.109681554, 'illustrator'),
  (-0.09397832, 'editor'),
  (-0.08254311, 'businesswoman'),
  (-0.08049533, 'welder'),
  (-0.07740901, 'butcher'),
  (-0.075390086, 'graphic_designer'),
  (-0.073243156, 'adjunct_professor'),
  (-0.07251743, 'interior_designer'),
  (-0.07244376, 'associate_professor'),
  (-0.07186047, 'alter_ego'),
  (-0.06930055, 'substitute'),
  (-0.06922661, 'drug_addict'),
  (-0.067476206, 'tutor'),
  (-0.06643469, 'homemaker'),
  (-0.06643169, 'hairdresser'),
  (-0.065213606, 'taxi_driver'),
  (-0.06506446, 'investment_banker')],
 [(0.08317426, 'aide'),
  (0.084812164, 'commander'),
  (0.091441356, 'minister'),
  (0.0929672, 'priest'),
  (0.09381424, 'politician'),
  (0.095800914, 'senator'),
  (0.097313896, 'citizen'),
  (0.09751704, 'housekeeper'),
  (0.099416405, 'assassin'),
  (0.100591816, 'goalkeeper'),
  (0.101061106, 'industrialist'),
  (0.10162376, 'trooper'),
